### Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#### Data Collection

In [3]:
movies= pd.read_csv('movies.csv')

In [4]:
movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [5]:
movies.shape

(4803, 24)

In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

### Data Preprocessing

In [7]:
movies.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

There are null values in the given dataset.

In [8]:
# selecting the relevant columns for recommendation

selected_columns = ['genres','keywords','tagline','cast','crew']

In [9]:
print(selected_columns)

['genres', 'keywords', 'tagline', 'cast', 'crew']


In [10]:
# Treating the null values 
for columns in selected_columns:
    movies[columns]=movies[columns].fillna("")



In [11]:
movies[columns].isna().sum()

0

No null values 

In [12]:
# combining the all the selected the columns for recommendation
combined_columns = movies['genres']+movies['keywords']+movies['tagline']+movies['cast']+movies['crew']

In [13]:
combined_columns

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

In [14]:
# converting the text data to vector format
from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer()

In [15]:
column_vectors = vector.fit_transform(combined_columns)

In [16]:
print(column_vectors)

  (0, 29020)	0.017914967565500414
  (0, 143309)	0.017914967565500414
  (0, 232134)	0.017914967565500414
  (0, 213445)	0.014097451891575812
  (0, 20865)	0.017914967565500414
  (0, 113885)	0.017914967565500414
  (0, 186626)	0.006966371169151836
  (0, 20864)	0.017088026863895583
  (0, 113884)	0.017914967565500414
  (0, 231629)	0.017088026863895583
  (0, 20863)	0.017914967565500414
  (0, 113883)	0.017914967565500414
  (0, 226601)	0.017914967565500414
  (0, 20862)	0.017088026863895583
  (0, 113881)	0.017914967565500414
  (0, 219711)	0.017088026863895583
  (0, 217723)	0.009856458491561158
  (0, 20861)	0.017914967565500414
  (0, 113879)	0.017914967565500414
  (0, 215342)	0.017914967565500414
  (0, 223223)	0.014097451891575812
  (0, 20860)	0.017914967565500414
  (0, 113876)	0.017914967565500414
  (0, 212972)	0.016046204790263332
  (0, 212133)	0.012933833232106909
  :	:
  (4802, 195955)	0.12162481200090412
  (4802, 202171)	0.13789948258900106
  (4802, 198453)	0.1260132015601203
  (4802, 217604)

In [17]:
column_vectors.shape

(4803, 233478)

In [18]:
# Getting the similarity score using cosine similarity 
similarity = cosine_similarity(column_vectors)

In [19]:
print(similarity)

[[1.         0.58026567 0.82572913 ... 0.36346082 0.10013839 0.22276774]
 [0.58026567 1.         0.58614356 ... 0.3271617  0.08722262 0.19737016]
 [0.82572913 0.58614356 1.         ... 0.38168384 0.10687146 0.22645475]
 ...
 [0.36346082 0.3271617  0.38168384 ... 1.         0.05998309 0.14005149]
 [0.10013839 0.08722262 0.10687146 ... 0.05998309 1.         0.03881615]
 [0.22276774 0.19737016 0.22645475 ... 0.14005149 0.03881615 1.        ]]


In [20]:
similarity.shape

(4803, 4803)

In [21]:
movie_name = input('Enter movie name: ')

Enter movie name: batman


In [22]:
list_of_all_titles=movies['title'].tolist()

In [23]:
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [24]:
# finding close match with the movie given by user
import difflib
find_close_match=difflib.get_close_matches(movie_name,list_of_all_titles)

In [25]:
print(find_close_match)

['Batman', 'Batman', 'Catwoman']


In [26]:
close_match=find_close_match[0]

In [27]:
close_match

'Batman'

In [28]:
# finding the index with the title
index_of_movie=movies[movies.title==close_match]['index'].values[0]

In [29]:
print(index_of_movie)

1359


In [30]:
# getting list of similar movies
similarity_score= list(enumerate(similarity[index_of_movie]))

In [31]:
print(similarity_score)

[(0, 0.3626251065884239), (1, 0.33230454938586407), (2, 0.37568619332406433), (3, 0.4039822277608021), (4, 0.38167913853994384), (5, 0.3634311377224157), (6, 0.3309052430611787), (7, 0.3807577729784404), (8, 0.3026137032454129), (9, 0.4047180977054302), (10, 0.3455401646477341), (11, 0.2705459446934927), (12, 0.2985266178957101), (13, 0.3232869524432485), (14, 0.370312392316433), (15, 0.3754662064826467), (16, 0.3626421302831715), (17, 0.3322850681110646), (18, 0.3033780427643775), (19, 0.37821362842677875), (20, 0.36518256760534606), (21, 0.2628018852408712), (22, 0.3785748087758161), (23, 0.32562853836174777), (24, 0.322801844835077), (25, 0.3501166491659437), (26, 0.3795166949684533), (27, 0.3726508532513787), (28, 0.39492864235497904), (29, 0.3165795391289812), (30, 0.40209481092988), (31, 0.32379717318223017), (32, 0.3277411077049453), (33, 0.34008605659032837), (34, 0.2379285341766641), (35, 0.36413901585526065), (36, 0.36418707640516806), (37, 0.32052585176619264), (38, 0.381532

In [32]:
len(similarity_score)

4803

In [33]:
sorted_movies=sorted(similarity_score, key=lambda x:x[1],reverse=True)

In [34]:
sorted_movies

[(1359, 1.0000000000000002),
 (303, 0.4124687752372974),
 (428, 0.41045905173709313),
 (119, 0.40977215057782634),
 (165, 0.40751329922339447),
 (291, 0.4057602553676514),
 (386, 0.4054501651287998),
 (186, 0.40533193302741016),
 (210, 0.4051559532865628),
 (112, 0.40487491652864366),
 (9, 0.4047180977054302),
 (93, 0.40463871173430055),
 (408, 0.40446068826123666),
 (320, 0.4044522567384423),
 (3, 0.4039822277608021),
 (673, 0.40321626608885186),
 (275, 0.4030710445236317),
 (150, 0.4021816703535511),
 (30, 0.40209481092988),
 (65, 0.40207184286010705),
 (197, 0.40207030725095977),
 (182, 0.4016340899736001),
 (935, 0.40153787737883734),
 (276, 0.40143928533804146),
 (529, 0.40095891771237097),
 (250, 0.4007457868143204),
 (191, 0.40068874007055577),
 (180, 0.4005584042188115),
 (164, 0.4005422814751241),
 (828, 0.4001486640183698),
 (263, 0.40010198792965124),
 (2743, 0.4000704625605317),
 (223, 0.3999472091792098),
 (171, 0.39963186712960086),
 (505, 0.3996257191885829),
 (185, 0.39

In [35]:
print('movies suggested for you: /n')

i=1
for movie in sorted_movies:
    index=movie[0]
    title_from_index=movies[movies.index==index]['title'].values[0]
    if(i<30):
        print(i,'.',title_from_index)
        i+=1
    

movies suggested for you: /n
1 . Batman
2 . Catwoman
3 . Batman Returns
4 . Batman Begins
5 . Hulk
6 . National Treasure
7 . The Bourne Supremacy
8 . Bad Boys II
9 . Batman & Robin
10 . Alexander
11 . Batman v Superman: Dawn of Justice
12 . Terminator 3: Rise of the Machines
13 . Collateral Damage
14 . Black Hawk Down
15 . The Dark Knight Rises
16 . Austin Powers in Goldmember
17 . Minority Report
18 . Men in Black II
19 . Spider-Man 2
20 . The Dark Knight
21 . Harry Potter and the Philosopher's Stone
22 . Ant-Man
23 . Herbie Fully Loaded
24 . Harry Potter and the Chamber of Secrets
25 . Tears of the Sun
26 . The Aviator
27 . Harry Potter and the Prisoner of Azkaban
28 . The Bourne Ultimatum
29 . Lethal Weapon 4


In [36]:
movie_name = input('Enter movie name: ')
list_of_all_titles=movies['title'].tolist()
find_close_match=difflib.get_close_matches(movie_name,list_of_all_titles)
close_match=find_close_match[0]
index_of_movie=movies[movies.title==close_match]['index'].values[0]
similarity_score= list(enumerate(similarity[index_of_movie]))
sorted_movies=sorted(similarity_score, key=lambda x:x[1],reverse=True)
print('movies suggested for you: /n')

i=1
for movie in sorted_movies:
    index=movie[0]
    title_from_index=movies[movies.index==index]['title'].values[0]
    if(i<30):
        print(i,'.',title_from_index)
        i+=1

Enter movie name: avatar
movies suggested for you: /n
1 . Avatar
2 . The Day After Tomorrow
3 . The Core
4 . The Avengers
5 . Captain America: The First Avenger
6 . Spectre
7 . The Curious Case of Benjamin Button
8 . Fast Five
9 . The Hunger Games: Mockingjay - Part 1
10 . Exodus: Gods and Kings
11 . Jurassic World
12 . Deep Blue Sea
13 . Blade II
14 . Dawn of the Planet of the Apes
15 . Ender's Game
16 . Terminator Genisys
17 . Cloud Atlas
18 . Contact
19 . The Hobbit: The Battle of the Five Armies
20 . The Girl with the Dragon Tattoo
21 . 2012
22 . Allegiant
23 . The X Files: I Want to Believe
24 . The Twilight Saga: Breaking Dawn - Part 2
25 . Cursed
26 . The Revenant
27 . In the Heart of the Sea
28 . The Amazing Spider-Man
29 . Alice Through the Looking Glass
